In [1]:
# read original data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Dataset_dir = "C:/Users/0stix/Datasets/bike_sharing_demand/"

df_orig_train = pd.read_csv(Dataset_dir+"train.csv")
df_orig_test = pd.read_csv(Dataset_dir+"test.csv")
df_orig_sample = pd.read_csv(Dataset_dir+"sampleSubmission.csv")

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEditor(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        



X_ = df_orig_train.drop(["casual", "registered", "count"], axis=1).copy()
y_ = df_orig_train["count"].copy()
y_log = np.log1p(y_)

X_["datetime"] = X_.apply(pd.to_datetime)
X_["year"] = X_.datetime.apply(lambda x: x.year)
X_["month"] = X_.datetime.apply(lambda x: x.month)
X_["day"] = X_.datetime.apply(lambda x: x.day)
X_["hour"] = X_.datetime.apply(lambda x: x.hour)
X_.drop("datetime", axis=1, inplace=True)
X_ = pd.get_dummies(X_, columns=["year", "month", "day", "hour", "holiday", "workingday", "season", "weather"])

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

X_train, X_val, y_train_log, y_val_log = train_test_split(X_, y_log, test_size=0.3, random_state=0)

lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train_log)
y_pred_log = lr_reg.predict(X_val)
y_pred = np.expm1(y_pred_log)
y_val = np.expm1(y_val_log)
evaluate_regr(y_val, y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred) ** 2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle


def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))


def evaluate_regr(y, pred):
    dct_ = {'rmsle_val': rmsle(y, pred),
            'rmse_val': rmse(y, pred),
            'mae_val': mean_absolute_error(y, pred)}
    print(dct_.items())